<a href="https://colab.research.google.com/github/BhyllionSolomon/Robotmato_Project/blob/main/tomatoclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Step 3: Define dataset path in Google Drive
dataset_path = "/content/drive/MyDrive/TomatoDataset"

# Step 4: Define image parameters
img_size = (150, 150)  # Resize all images to 150x150
batch_size = 32  # Number of images processed at a time

# Step 5: Data augmentation & preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values (0 to 1)
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% train, 20% validation
)

# Step 6: Load training data
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",  # Two classes: Ripe, Rotten
    subset="training"
)

# Step 7: Load validation data
val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)

# Step 8: Define CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),  # Prevent overfitting
    tf.keras.layers.Dense(1, activation="sigmoid")  # Output layer (Ripe or Rotten)
])

# Step 9: Compile model
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Step 10: Train model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10  # You can increase epochs for better accuracy
)

# Step 11: Save trained model to Google Drive
model_save_path = "/content/drive/MyDrive/TomatoDataset/tomato_model.h5"
model.save(model_save_path)
print("Model saved successfully at:", model_save_path)


Mounted at /content/drive
Found 1288 images belonging to 1 classes.
Found 322 images belonging to 1 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 251s 6s/step - accuracy: 0.9262 - loss: 0.0837 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 1.0

Model saved successfully at: /content/drive/MyDrive/TomatoDataset/tomato_model.h5


In [8]:
!pip install flask flask-ngrok tensorflow numpy


In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import tensorflow as tf
import cv2
import base64
import io
from PIL import Image
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Enable ngrok to access Flask app

# Load the trained model
model = tf.keras.models.load_model("/content/drive/MyDrive/TomatoDataset/tomato_model.h5", compile=False)
class_labels = ["Ripe Tomato", "Rotten Tomato"]

def preprocess_image(image_data):
    """ Convert base64 image data to a NumPy array and preprocess it. """
    image = Image.open(io.BytesIO(image_data)).convert("RGB")
    image = image.resize((150, 150))  # Resize to match model input
    image = np.array(image) / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)
    return image

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json['image']
        image_data = base64.b64decode(data.split(",")[1])  # Decode base64
        image = preprocess_image(image_data)  # Preprocess image

        prediction = model.predict(image)
        label = class_labels[np.argmax(prediction)]

        return jsonify({"prediction": label})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-21:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           